## Data Info
Getting the data info such as data size, number of feature, number of records, mean value, max value, etc....

In [2]:
import pandas as pd

In [3]:
crime_data = pd.read_csv('Crimes_2001_to_Present.csv')

In [4]:
crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000075 entries, 0 to 8000074
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Case Number           object 
 2   Date                  object 
 3   Block                 object 
 4   IUCR                  object 
 5   Primary Type          object 
 6   Description           object 
 7   Location Description  object 
 8   Arrest                bool   
 9   Domestic              bool   
 10  Beat                  int64  
 11  District              float64
 12  Ward                  float64
 13  Community Area        float64
 14  FBI Code              object 
 15  X Coordinate          float64
 16  Y Coordinate          float64
 17  Year                  int64  
 18  Updated On            object 
 19  Latitude              float64
 20  Longitude             float64
 21  Location              object 
dtypes: bool(2), float64(7), int64(3), object(1

## Converting the Arrest Feature data type
bool -> int

In [19]:
crime_data['Arrest'] = crime_data['Arrest'].replace({
    True:1,
    False:0
})

crime_data['Arrest'].unique()

array([0, 1])

## Converting the Date


In [5]:

crime_data['New_Date'] = pd.to_datetime(crime_data['Date'])
# crime_data['New_Date'] = crime_data['New_Date'].dt.strftime('%m-%d-%Y %H:%M:%S')

In [6]:
# comparing the old date value format to the datetime value format
crime_data[['Date', 'New_Date']]

,Date,New_Date
0,03/18/2015 12:00:00 PM,2015-03-18 12:00:00
1,12/20/2018 03:00:00 PM,2018-12-20 15:00:00
2,05/01/2016 12:25:00 AM,2016-05-01 00:25:00
3,12/20/2018 04:00:00 PM,2018-12-20 16:00:00
4,06/01/2014 12:01:00 AM,2014-06-01 00:01:00
...,...,...
8000070,08/10/2020 08:30:00 PM,2020-08-10 20:30:00
8000071,06/18/2020 09:00:00 PM,2020-06-18 21:00:00
8000072,07/27/2020 03:02:00 PM,2020-07-27 15:02:00
8000073,08/14/2020 03:00:00 PM,2020-08-14 15:00:00


In [7]:
print(crime_data[crime_data['New_Date'].dt.year >= 2021].count()) # How many records we have from 2021 to 2024
print("Max new_date value: ", crime_data['New_Date'].max()) # Latest record of 2024
print("Min new_date value: ", crime_data['New_Date'].min()) # Earliest record of 2021

ID                      734075
Case Number             734075
Date                    734075
Block                   734075
IUCR                    734075
Primary Type            734075
Description             734075
Location Description    730589
Arrest                  734075
Domestic                734075
Beat                    734075
District                734075
Ward                    734051
Community Area          734074
FBI Code                734075
X Coordinate            723460
Y Coordinate            723460
Year                    734075
Updated On              734075
Latitude                723460
Longitude               723460
Location                723460
New_Date                734075
dtype: int64
Max new_date value:  2024-02-10 00:00:00
Min new_date value:  2001-01-01 00:00:00


In [8]:
crime_data = crime_data[crime_data['New_Date'].dt.year >= 2021]

In [9]:
print("Min new_date value: ", crime_data['New_Date'].min()) # Earliest record of 2021
print(crime_data['New_Date'].dt.year.unique()) # Making sure that the range (2021-2024)

Min new_date value:  2021-01-01 00:00:00
[2023 2021 2022 2024]


In [14]:

crime_data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,New_Date
371,13204489,JG416325,09/06/2023 11:00:00 AM,0000X E 8TH ST,0810,THEFT,OVER $500,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,...,32.0,06,1176857.0,1896680.0,2023,11/04/2023 03:40:18 PM,41.871835,-87.626151,"(41.871834768, -87.62615082)",2023-09-06 11:00:00
643,12342615,JE202211,04/17/2021 03:20:00 PM,081XX S PRAIRIE AVE,0325,ROBBERY,VEHICULAR HIJACKING,RESIDENCE,True,False,...,44.0,03,1179448.0,1851073.0,2021,09/14/2023 03:41:59 PM,41.746626,-87.618032,"(41.746626309, -87.618031954)",2021-04-17 15:20:00
646,12589893,JF109865,01/11/2022 03:00:00 PM,087XX S KINGSTON AVE,1565,SEX OFFENSE,INDECENT SOLICITATION OF A CHILD,RESIDENCE,False,True,...,46.0,17,1194660.0,1847481.0,2022,09/14/2023 03:41:59 PM,41.736409,-87.562410,"(41.736409029, -87.562410309)",2022-01-11 15:00:00
647,12592454,JF113025,01/14/2022 03:55:00 PM,067XX S MORGAN ST,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,RESIDENCE,False,True,...,68.0,26,1170805.0,1860170.0,2022,09/14/2023 03:41:59 PM,41.771782,-87.649437,"(41.771782439, -87.649436929)",2022-01-14 15:55:00
648,12785595,JF346553,08/05/2022 09:00:00 PM,072XX S UNIVERSITY AVE,1544,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,APARTMENT,True,False,...,69.0,17,1185135.0,1857211.0,2022,09/14/2023 03:41:59 PM,41.763338,-87.597001,"(41.763337967, -87.597001131)",2022-08-05 21:00:00


## Columns that need to drop
X & Y Coordinate, Date, Block, IUCR, Description, Domestic, Beat, District, FBI code, Ward, Updated on, Latitude, Longitude 

In [20]:
col = ['ID', 'New_Date', 'Primary Type', 'Location Description', 'Arrest', 'Community Area', 'Location']
crime_data = crime_data[col]
crime_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 734075 entries, 371 to 7264756
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   ID                    734075 non-null  int64         
 1   New_Date              734075 non-null  datetime64[ns]
 2   Primary Type          734075 non-null  object        
 3   Location Description  730589 non-null  object        
 4   Arrest                734075 non-null  int64         
 5   Community Area        734074 non-null  float64       
 6   Location              723460 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 44.8+ MB


## Saving the Dataframe to a CSV file

In [21]:
crime_data.to_csv('Crimes_2021_to_Present.csv')

In [22]:
print("hello worlkd")

hello worlkd
